# Creating Categorical Encoders

In [1]:
import sys

sys.path.append("/home/jupyter-aamir09/mlops2_with_dagster/")

In [2]:
from mlops2_with_dagster.utils import get_project_dir, printse
project = 'mlops2_with_dagster'
project_dir = get_project_dir(project)
printse(f'project_dir: {project_dir}')

>>>> project_dir: /home/jupyter-aamir09/mlops2_with_dagster


In [3]:
import pandas as pd
from joblib import dump, load
from hamilton import driver, base

In [4]:
from mlops2_with_dagster import encoder_pipeline

In [5]:
from pathlib import Path
train_data : Path = project_dir/"data/train.csv"
test_data : Path = project_dir/"data/test.csv"

In [6]:
# parameters
df_train = pd.read_csv(train_data)
df_test = pd.read_csv(test_data)

In [7]:
index_col = 'passengerid'
target_col = "survived"
cat_cols = ["sex", "cabin", "embarked"]
config = {
    'index_column': index_col,
    'target_column': target_col,
    'categorical_columns': cat_cols
}

In [8]:
printse(pd.__version__)

>>>> 2.0.3


In [9]:
adapter = base.SimplePythonGraphAdapter(base.DictResult())
encode_dr = driver.Driver(config, encoder_pipeline, adapter=adapter)
output_nodes = ['encoders']
# encode_dr.visualize_execution(output_nodes, project_dir/'artifacts/encoder.dot', {}, 
#     inputs = dict(
#         df_train = df_train,
#         df_test = df_test
#     )                           
# )

Note: Hamilton collects completely anonymous data about usage. This will help us improve Hamilton over time. See https://github.com/dagworks-inc/hamilton#usage-analytics--data-privacy for details.


In [10]:
out = encode_dr.execute(['encoders'],
        inputs = dict(
            df_train = df_train,
            df_test = df_test
        )         
)
out

>>>> Index(['sex', 'cabin', 'embarked'], dtype='object')
>>> <class 'pandas.core.series.Series'> 200000 ['C12239' nan nan nan nan nan nan nan 'A7253' 'D2969' nan 'A9055' 'C8674'
 nan 'D6945' nan nan nan nan 'B7010' nan nan nan nan nan 'B11790' nan
 'C20297' nan nan nan nan nan 'B16390' nan nan 'C19277' 'B7665' nan nan
 'D6224' nan 'B21244' nan nan nan nan 'D1695' nan nan 'C12615' nan
 'D19734' 'C11880' nan nan 'A19055' 'D6286' nan nan nan nan nan 'A6932'
 nan nan 'C15652' 'B4480' 'C5522' nan 'A19030' nan nan 'D19747' nan nan
 nan nan nan nan nan nan 'B19226' nan nan nan nan nan nan 'A18210' nan
 'B5371' nan nan 'C11610' nan nan 'D7415' nan 'C16097']


{'encoders': {'cabinencoder': LabelEncoder(),
  'sexencoder': LabelEncoder(),
  'embarkedencoder': LabelEncoder()}}

In [11]:
import dagstermill

dagstermill.yield_result(out, output_name="encoders")

{'encoders': {'cabinencoder': LabelEncoder(),
  'sexencoder': LabelEncoder(),
  'embarkedencoder': LabelEncoder()}}